In [4]:
import numpy as np
import pandas as pd 
import os
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,StratifiedKFold,RepeatedStratifiedKFold
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [5]:
train_df      = pd.read_csv('archive/train.csv')
test_df       = pd.read_csv('archive/test.csv')
submission_df = pd.read_csv('archive/sample_submission.csv')

In [6]:
#Creating Addition Features
combine_set=pd.concat([train_df,test_df],axis=0)
combine_set['City_Code_Patient'].fillna(-99,inplace=True)
combine_set['Bed Grade'].fillna(5,inplace=True)
combine_set['Unique_Hospital_per_patient']=combine_set.groupby(['patientid'])['Hospital_code'].transform('nunique')
combine_set['Unique_patient_per_hospital']=combine_set.groupby(['Hospital_code'])['patientid'].transform('nunique')
combine_set['Unique_patient_per_Department']=combine_set.groupby(['Department'])['patientid'].transform('nunique')
combine_set['Total_deposit_paid_by_patient_in_each_hospital']=combine_set.groupby(['Hospital_code','patientid'])['Admission_Deposit'].transform('sum')
combine_set['Min_Severity_of_Illness'] = combine_set.groupby('patientid')['Severity of Illness'].transform('min')

In [7]:

le = LabelEncoder()
#Encoding categorical variables by frequency encoding and label encoding
for col in combine_set.select_dtypes(include='object').columns:
    if col not in ['Age','Stay']:
        fe=combine_set.groupby([col]).size()/len(combine_set)
        combine_set[col]=combine_set[col].apply(lambda x: fe[x])   
        # combine_set[col]  = pd.get_dummies(combine_set[col].astype(str))         
    elif col!='Stay':
        combine_set[col]=le.fit_transform(combine_set[col].astype(str))
    else:
        pass


In [8]:
#Splitting train and test

X=combine_set[combine_set['Stay'].isnull()==False].drop(['case_id','Stay'],axis=1)
y=le.fit_transform(combine_set[combine_set['Stay'].isnull()==False]['Stay'])
y=pd.DataFrame(y,columns=['Stay'])
X_main_test=combine_set[combine_set['Stay'].isnull()==True].drop(['case_id','Stay'],axis=1)

In [9]:
# kf=KFold(n_splits=10,shuffle=True,random_state=294)
kf=KFold(n_splits=5,shuffle=True)

preds_1   = {}
y_pred_1  = []
acc_score = []

for i,(train_idx,val_idx) in enumerate(kf.split(X)):    
    
    X_train, y_train = X.iloc[train_idx,:], y.iloc[train_idx]
    
    X_val, y_val = X.iloc[val_idx, :], y.iloc[val_idx]

    print('\nFold: {}\n'.format(i+1))




Fold: 1


Fold: 2


Fold: 3


Fold: 4


Fold: 5



In [9]:

num_classes = len(np.unique(y_train))
print("Nombre de classes réelles :", num_classes)

Nombre de classes réelles : 11


In [10]:


# Vérifier le type de y_train
print("Type de y_train :", type(y_train))

# Vérifier les valeurs uniques dans y_train
unique_classes = np.unique(y_train)
print("Classes réelles :", unique_classes)
print("Nombre de classes réelles :", len(unique_classes))

Type de y_train : <class 'pandas.core.frame.DataFrame'>
Classes réelles : [ 0  1  2  3  4  5  6  7  8  9 10]
Nombre de classes réelles : 11


In [18]:
lg=LGBMClassifier(device="gpu", 
                      gpu_platform_id= 0,
                      max_bin=63,#Theoretically best speeds using LGBM
                      gpu_device_id= 0,
                      boosting_type='gbdt',
                      learning_rate=0.04,
                      # max_depth=15,
                      # num_leaves = 150,
                      objective='multi_class',
                      num_class=11,                      
                      n_estimators=50000,
                      metric='multi_error',
                      colsample_bytree=0.8,
                      min_child_samples=228,
                      reg_alpha=1,
                      reg_lambda=1,
                      # random_state=294,
                      n_jobs=-1,

                       ) 

In [ ]:
 
# cette entrainement ne marche pas  ma il me faut update mon Classifier

lg.fit(X_train, y_train
    #,categorical_feature = categorical_features
    ,eval_metric='multi_error'
    ,eval_set=[(X_train, y_train),(X_val, y_val)]
    ,early_stopping_rounds=100
    ,verbose=50
)




In [11]:
# Marche
import lightgbm as lgb

# Préparation des données d'entraînement X_train et y_train

#  les paramètres du modèle
#diminuer le nombre de parametre n_estimators sa peut cracher ma machine
params = {
    'max_bin':63,
    'boosting_type':'gbdt',
    'learning_rate':0.04,
    'objective': 'multiclass',
    'metric': ['multi_logloss'],
    'num_class': 11,
    'n_estimators':5000,
    'metric':'multi_error',
    'colsample_bytree':0.8,
    'min_child_samples':228,
    'reg_alpha':1,
    'reg_lambda':1,
    'n_jobs':-1,
      
    # Autres paramètres du modèle
}

# Création d'un dataset LightGBM à partir des données d'entraînement
train_data = lgb.Dataset(X_train, label=y_train)

# Entraînement du modèle
model = lgb.train(params, train_data)

# Utilisation le modèle pour faire des prédictions donnee de test

predictions = model.predict(X_val)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033567 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data points in the train set: 254751, number of used features: 21
[LightGBM] [Info] Start training from score -2.600490
[LightGBM] [Info] Start training from score -1.404432
[LightGBM] [Info] Start training from score -1.291491
[LightGBM] [Info] Start training from score -1.753011
[LightGBM] [Info] Start training from score -3.297876
[LightGBM] [Info] Start training from score -2.209512
[LightGBM] [Info] Start training from score -4.760503
[LightGBM] [Info] Start training from score -3.443865
[LightGBM] [Info] Start training from score -4.186257
[LightGBM] [Info] Start training from score -4.742779
[LightGBM] [Info] Start tra

In [12]:
#affichage des predictions
print(predictions)

[[1.57825128e-02 2.25571592e-01 4.29950352e-01 ... 2.97488538e-04
  7.75231014e-04 1.21426089e-04]
 [2.93246145e-02 2.72643411e-01 3.91421628e-01 ... 9.77374230e-05
  3.29480386e-05 9.66433672e-05]
 [3.50256365e-02 1.06568212e-01 2.42831215e-01 ... 1.39432647e-02
  3.80074392e-04 7.12770460e-04]
 ...
 [8.08279297e-05 1.04924945e-01 4.73300094e-03 ... 5.82868862e-04
  3.39215726e-03 8.80690326e-03]
 [1.29867679e-03 3.41108657e-02 2.93826342e-02 ... 1.82648142e-03
  1.27520189e-02 2.63779959e-03]
 [1.67089500e-03 8.50772279e-02 5.72075848e-02 ... 4.75330715e-03
  2.12118003e-02 2.04017092e-03]]


In [13]:
from sklearn.metrics import classification_report

# Convertir les prédictions en classes
y_pred_class = [np.argmax(pred) for pred in predictions]

# Calculer le rapport de classification
report = classification_report(y_val, y_pred_class)

# Afficher le rapport de classification
print(report)

              precision    recall  f1-score   support

           0       0.39      0.22      0.28      4692
           1       0.43      0.48      0.46     15596
           2       0.43      0.64      0.52     17470
           3       0.39      0.25      0.31     11023
           4       0.16      0.01      0.03      2327
           5       0.42      0.48      0.45      7058
           6       0.31      0.02      0.03       563
           7       0.36      0.12      0.17      2117
           8       0.40      0.24      0.30       965
           9       0.44      0.10      0.16       545
          10       0.60      0.50      0.55      1331

    accuracy                           0.43     63687
   macro avg       0.39      0.28      0.30     63687
weighted avg       0.41      0.43      0.40     63687



In [ ]:
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve, auc


# Calculer les courbes ROC ne se trace pas pour les probleme de classification multiclasse uniquement binaire mais on peut faire plusieurs courbe

# fpr, tpr, _ = roc_curve(X_val, y_val)
# roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC

# plt.figure()
# plt.plot(fpr, tpr, color='darkorange', lw=2, label='Courbe ROC (AUC = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('Taux de faux positifs')
# plt.ylabel('Taux de vrais positifs')
# plt.title('Courbe ROC')
# plt.legend(loc="lower right")
# plt.show()

# Calculer les courbes de précision-rappel

precision, recall, _ = precision_recall_curve(X_val, y_val)

# Tracer la courbe de précision-rappel
plt.figure()
plt.plot(recall, precision, color='blue', lw=11, label='Courbe de précision-rappel')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Rappel')
plt.ylabel('Précision')
plt.title('Courbe de précision-rappel')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score


print(accuracy_score(y_val,model.predict(X_val))*100)
    
acc = accuracy_score(y_val,model.predict(X_val))*100
acc_score.append(acc)
print("Score : ",acc)    
y_pred_1.append(model.predict_proba(X_main_test))
    


# preds_1[i+1]=lg.predict_proba(X_main_test)
# y_pred_1.append(lg.predict_proba(X_main_test))


In [ ]:
y_pred_final_1 = np.mean(np.array(y_pred_1),axis=0)
    
print('mean accuracy score: {}'.format((sum(acc_score)/10)))

preds_1=np.argmax(y_pred_final_1,axis=1)

print(preds_1.shape)
submission_df['Stay']=le.inverse_transform(preds_1.astype(int))
# submission_df[0] = y_pred_final_1[:,0]
# submission_df[1] =y_pred_final_1[:,1]

# Download Submission File :
display("submission_df",submission_df)
sub_file_name_1 = "BEST_11_CV=42.96_LB=WAIT_LGBM-1.csv"


submission_df.to_csv(sub_file_name_1,index=False)
submission_df.head(5)